## A notebook to collate data from Dimensions Google BigQuery database

### Last updated: 19th January 2023

#### Authors: Charlie Rahal and Saurabh Khanna

Load dependancies:

In [1]:
import pandas as pd
import os
from google.cloud import bigquery
from helper_functions import get_pubs_all_issn

Authenticate with to the bigquery client and project:

In [2]:
MY_PROJECT_ID = "dimensionspkp"
client = bigquery.Client(project=MY_PROJECT_ID)

Load the raw list of issns

In [3]:
raw_path = os.path.join("..", "data", "raw")
raw_data = pd.read_csv(os.path.join(raw_path, "ojs_issns_in_dimensions_2020.csv"))
issns_to_query = raw_data["issn"].tolist()
print("Total ISSNs to query: ", len(issns_to_query))

Total ISSNs to query:  12435


Set a raw dimensions output path:

In [ ]:
dim_out = os.path.join('..', 'data', 'raw', 'from_dimensions')

Call our helper function to grab the data for all pubs in our ISSN list. Note: we chunk into 1000 issns to get over local memory file limits.

In [ ]:
%%time
file_name = 'pubs_from_all_issns.csv'
file_path = os.path.join(dim_out, file_name)
get_pubs_all_issn(1000, file_path, client, issns_to_query)

9it [56:03, 421.33s/it]

Lets now get a list of all unique publications found in our raw data returned from the above function calls